## Cash Flow Data Generation

In [1]:
import numpy as np
import pandas as pd

np.random.seed(101)

## Business Parameters

I'm simulating 10 years (120 months) of cash flow for a mid-sized company and I try to make the sceanrio as real as possible

**Starting position :** 100k cash  (This will work as a cushion)                         
**Average revenue :** 50k per month with seasonal changes  
**Cost structure :** 50% COGS, $15k fixed costs, 10% variable costs

I also built in two events for making this simulation more realistic :
- A recession in years 5 and 6 (revenue drops by 20%)
- A policy change in year 7 (faster payment collection)

In [2]:
months = 120
cash_initial = 100000
avg_revenue = 50000
seasonal_factor = [1.1, 1.0, 0.9, 0.95, 1.2, 1.15, 1.0, 0.9, 0.85, 1.05, 1.2, 1.1] * 10
COGS_ratio = 0.5
OPEX_fixed = 15000
OPEX_variable_ratio = 0.1
AR_Delay_options = [0, 1, 2, 3]

# Recession: revenue drops 20% in years 5 and 6
recession_years = [5, 6]
# Policy Change: After year 7, payment delays improve
policy_change_year = 7

## Generate Revenue and Costs

Revenue changes with seasons and some random variation  
During recession (years 5-6), revenue drops by 20%

Payment delays are random between 0-3 months, but will improve after year 7

In [3]:
data = pd.DataFrame({
    'Month': range(1, months+1),
    'Year': [(i//12)+1 for i in range(months)]
})

# Generate revenue with seasonality
revenue_list = []
for sf in seasonal_factor:
    rev = avg_revenue * sf * np.random.uniform(0.9, 1.1)
    revenue_list.append(rev)
data['Revenue'] = revenue_list

# Apply recession
data['Recession'] = 0
for year in recession_years:
    data.loc[data['Year'] == year, 'Recession'] = 1
    data.loc[data['Year'] == year, 'Revenue'] = data.loc[data['Year'] == year, 'Revenue'] * 0.8

# AR_Delay
data['AR_Delay'] = np.random.choice(AR_Delay_options, size=months)
data.loc[0, 'AR_Delay'] = 0  # First month has no delay

# Policy Change: After year 7, delays get better
for i in range(len(data)):
    if data.loc[i, 'Year'] >= policy_change_year:
        if data.loc[i, 'AR_Delay'] > 2:
            data.loc[i, 'AR_Delay'] = 2

# Other Expenses
data['Other_Expenses'] = np.random.randint(0, 3000, size=months)

# Calculate COGS and OPEX
data['COGS'] = data['Revenue'] * COGS_ratio
data['OPEX_Variable'] = data['Revenue'] * OPEX_variable_ratio
data['OPEX_Fixed'] = OPEX_fixed
data['Total_Expenses'] = data['COGS'] + data['OPEX_Variable'] + data['OPEX_Fixed'] + data['Other_Expenses']

## Cash Collection with Payment Delays

When AR_Delay = 2, the cash from this month sales comes in 2 months later

This create a gap between revenue and actual cash received

In [4]:
# Calculate cash collected based on delay
cash_collected = []
for i in range(months):
    delay = int(data.loc[i, 'AR_Delay'])
    
    if delay == 0:
        collected = data.loc[i, 'Revenue']
    elif delay == 1 and i >= 1:
        collected = data.loc[i-1, 'Revenue']
    elif delay == 2 and i >= 2:
        collected = data.loc[i-2, 'Revenue']
    elif delay == 3 and i >= 3:
        collected = data.loc[i-3, 'Revenue']
    else:
        collected = 0
    
    cash_collected.append(collected)

data['Cash_Collected'] = cash_collected

# Calculate net cash flow
data['Net_Cash_Flow'] = (data['Cash_Collected'] - data['Total_Expenses']).round(0)

## Calculate Running Cash Balance

Track cash month by month : Cash_End = Cash_Start + Net_Cash_Flow

I flag months when cash drops below 10% of starting capital as a warning

In [5]:
cash_start_list = []
cash_end_list = []
shortage_alert_list = []

current_cash = cash_initial

for i in range(months):
    # Store starting cash
    cash_start_list.append(current_cash)
    
    # Calculate ending cash
    net_flow = data.loc[i, 'Net_Cash_Flow']
    current_cash = current_cash + net_flow
    cash_end_list.append(current_cash)
    
    # Check if shortage
    if current_cash < cash_initial * 0.1:
        shortage_alert_list.append(1)
    else:
        shortage_alert_list.append(0)

data['Cash_Start'] = cash_start_list
data['Cash_End'] = cash_end_list
data['Shortage_Alert'] = shortage_alert_list

In [6]:
print(data.head(12))

    Month  Year       Revenue  Recession  AR_Delay  Other_Expenses  \
0       1     1  55180.384905          0         0            1017   
1       2     1  50706.675869          0         2            1143   
2       3     1  40756.268038          0         3             788   
3       4     1  44379.455734          0         2            1013   
4       5     1  62223.323780          0         3            1865   
5       6     1  61339.813920          0         1            2107   
6       7     1  48069.662197          0         1            2594   
7       8     1  48542.517717          0         3             502   
8       9     1  44383.122825          0         2             453   
9      10     1  49244.359019          0         1            1659   
10     11     1  60650.731093          0         2            2766   
11     12     1  53373.451494          0         0            2012   

            COGS  OPEX_Variable  OPEX_Fixed  Total_Expenses  Cash_Collected  \
0   27590.

In [7]:
data.to_csv('simulated_cashflow_data.csv', index=False)